# Text Classification Task
In this task, you would require to claasify the BBC News text into 5 classes ['business' 'entertainment' 'politics' 'sport''tech']. For this task, the code skeleton has been given and you have to write your code below #TODO parts. Comments are given with #TODO as helpers

## Importing relevant libraries 
If any of the below list libraries is not installed already, then use "pip install #library_name" to install it

In [41]:
!pip install torch==1.6.0

In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Importing BBC News Dataset
Source data from public data set on BBC news articles:
D. Greene and P. Cunningham. "Practical Solutions to the Problem of Diagonal Dominance in Kernel Document Clustering", Proc. ICML 2006. [PDF] [BibTeX].

http://mlg.ucd.ie/datasets/bbc.html

Cleaned up version of the Dataset is given as csv file with the assignment

In [43]:
data_train = pd.read_csv("bbc-text_train.csv")
data_test= pd.read_csv("bbc-text_test.csv")

In [44]:
data_train.head()

,category,text
0,entertainment,farrell due to make us tv debut actor colin fa...
1,business,china continues rapid growth china s economy h...
2,business,ebbers aware of worldcom fraud former worldc...
3,entertainment,school tribute for tv host carson more than 1 ...
4,tech,broadband fuels online expression fast web acc...


In [45]:
data_train['category'].value_counts()

sport            413
business         409
politics         334
tech             319
entertainment    305
Name: category, dtype: int64

## Splitting training data into Train and validation set
Note: Validation set is surrogate to test set and while training the network , we evaluate the model on validation set

In [46]:
train_x_df,val_x_df,train_y_df,val_y_df = train_test_split(data_train['text'],data_train['category'],test_size=0.2,random_state=42)

## Encoding prediction classes/labels into integers


In [47]:
le.fit(train_y_df)
print(le.classes_)
train_y=le.transform(train_y_df)
val_y=le.transform(val_y_df)
test_y=le.transform(data_test['category'])

['business' 'entertainment' 'politics' 'sport' 'tech']


## Converting News text into numerical vector using count vectorizer

In [48]:
vectorizer = CountVectorizer()
vectorizer.fit_transform(train_x_df)
train_x=vectorizer.transform(train_x_df)
val_x=vectorizer.transform(val_x_df)
test_x=vectorizer.transform(data_test['text'])
input_dim = len(test_x.toarray()[1]) #24295

In [49]:
train_x.toarray()
print(len(test_x.toarray()[1]))

24295


In [50]:
class ClassificationNet(nn.Module):

    def __init__(self):
        super(ClassificationNet, self).__init__()
        '''
        Defining layers of neural network
        '''
        #TODO 1
        # input dim = 24295 = len(test_x.toarray()[1])
        # output dim = 5 = nr of categories)
        self.fc1 = nn.Linear(24295, 5)


    def forward(self, x):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (data_points, num_features)
        Returns:
            the resulting tensor.
        """
        #TODO 2
        x = torch.sigmoid(self.fc1(x))
        return x


net =ClassificationNet()

# TODO 3 (define learning rate)
w = 0.001


# TODO 4 (Construct/define an optimizer object)
#optimizer similiar "adaptive gradient descent" 
optimizer = optim.Adam(net.parameters(), lr=0.05)

# TODO 5 (Construct an loss/criterion object)
criterion = nn.BCELoss() #BCELoss() is suited if classifie 0 or 1

#TODO 6 (define number of epochs/ number of training iteration)
epochs= 30

#converting train and validation set arrays to tensor
train_x_tensor=torch.tensor(train_x.toarray()).float()
train_y_tensor=torch.tensor(train_y)
val_x_tensor=torch.tensor(val_x.toarray()).float()
val_y_tensor=torch.tensor(val_y)


def evaluation_metrics(predict_y,ground_truth_y):
    '''
    Returns accuracy and f1 score metrics for evaluation
    '''
    accuracy=accuracy_score(ground_truth_y,predict_y)
    f1score=f1_score(ground_truth_y,predict_y,average='macro')
    
    return accuracy,f1score

## Training Loop

In [51]:
for i in range(epochs):
    
    # the training routine is these 5 steps:
    
    # TODO 7 step 1. zero the gradients
    optimizer.zero_grad()
    
    # TODO 8 step 2. compute the output
    output = net(train_x_tensor)
    
    
    # TODO 9 step 3. compute the loss (name this as "loss")
    #given y-val (category 0-4) construct a vector(1,5) initalized to 0, then set vector[y-val] = 1
    #add the vectors to an array 
    #transform array to tensor
    train_y_adapted = []
    for ele in train_y:
        ele_res = [0,0,0,0,0]
        ele_res[ele] = 1
        train_y_adapted.append(ele_res) 
    
    train_y_adapted_tensor = torch.tensor(train_y_adapted)
    loss = criterion(output, train_y_adapted_tensor.float())
    
    # TODO 10 use loss to produce gradients
    #calc gardients
    loss.backward()
    
    # TODO 11 use optimizer to take gradient step
    optimizer.step() 
    
    
    with torch.no_grad():
        # validation set evaluation:
        
        # TODO 11 compute the output
        output_val = net(val_x_tensor)

        
        # TODO 12 compute the loss (name this as "loss_val")
        #given y-val (category 0-4) construct a vector(1,5) initalized to 0, then set vector[y-val] = 1
        #add the vectors to an array 
        #transform array to tensor
        val_y_adapted = []
        for ele in val_y:
            ele_res = [0,0,0,0,0]
            ele_res[ele] = 1
            val_y_adapted.append(ele_res) 

        val_y_adapted_tensor = torch.tensor(val_y_adapted)
        loss_val = criterion(output_val, val_y_adapted_tensor.float())
        
        # TODO 13 compute the prediction
        # take as prediction the category which has the hight value
        prediction = []
        for idx, val1 in enumerate(output_val):
            max = -1
            max_pos = 0
            for idx2, ele in enumerate(val1):
                if ele > max:
                    max = ele
                    max_pos = idx2
            prediction.append(max_pos)
            
        
        # TODO 14 Use the "evaluation_metrics" function to find accuracy and f1 score and name this as               "accuracy","f1score")
        (accuracy, f1score) = evaluation_metrics(val_y_tensor, prediction)

        
        
        print('Epoch %d/%d - Loss_train: %.3f   loss_val: %.3f   accuracy_val: %.3f f1score_val: %.3f   '% \
            (i + 1, epochs,loss.item(),loss_val.item(),accuracy,f1score))

Epoch 1/30 - Loss_train: 0.703   loss_val: 2.151   accuracy_val: 0.980 f1score_val: 0.979   
Epoch 2/30 - Loss_train: 2.067   loss_val: 1.773   accuracy_val: 0.949 f1score_val: 0.945   
Epoch 3/30 - Loss_train: 1.429   loss_val: 0.777   accuracy_val: 0.935 f1score_val: 0.929   
Epoch 4/30 - Loss_train: 0.370   loss_val: 0.268   accuracy_val: 0.938 f1score_val: 0.933   
Epoch 5/30 - Loss_train: 0.072   loss_val: 0.116   accuracy_val: 0.947 f1score_val: 0.942   
Epoch 6/30 - Loss_train: 0.023   loss_val: 0.108   accuracy_val: 0.949 f1score_val: 0.946   
Epoch 7/30 - Loss_train: 0.031   loss_val: 0.157   accuracy_val: 0.952 f1score_val: 0.949   
Epoch 8/30 - Loss_train: 0.060   loss_val: 0.309   accuracy_val: 0.952 f1score_val: 0.949   
Epoch 9/30 - Loss_train: 0.089   loss_val: 0.346   accuracy_val: 0.961 f1score_val: 0.958   
Epoch 10/30 - Loss_train: 0.089   loss_val: 0.398   accuracy_val: 0.963 f1score_val: 0.962   
Epoch 11/30 - Loss_train: 0.060   loss_val: 0.386   accuracy_val: 0.9

## Test set Prediction and Evaluation

In [52]:
# converting test set arrays to tensor
test_x_tensor=torch.tensor(test_x.toarray()).float()
test_y_tensor=torch.tensor(test_y)

with torch.no_grad():
    # Test set evaluation:
    
    # TODO 15 compute the output
    output_test = net(test_x_tensor)
    
    # TODO 16 compute the prediction
    prediction = []
    for idx, val1 in enumerate(output_test):
        max = -1
        max_pos = 0
        for idx2, ele in enumerate(val1):
            if ele > max:
                max = ele
                max_pos = idx2
        prediction.append(max_pos)
    
    # TODO 17 Use the "evaluation_metrics" function to find accuracy and f1 score
    (accuracy_test, f1score_test) = evaluation_metrics(test_y_tensor, prediction)
    
    print('Accuracy_test: %.3f f1score_val: %.3f   '% (accuracy_test,f1score_test))

Accuracy_test: 0.971 f1score_val: 0.971   
